### Library

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import datacompy
import os, sys
import numpy as np
import re
import ast
import gower

# narzedzia
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    learning_curve,
    RepeatedStratifiedKFold,
    GridSearchCV,
)
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score, accuracy_score, pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from scipy import stats
from scipy.stats import chi2_contingency
from scipy.spatial import distance
from joblib import dump, load

# modele
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# methods
from imblearn.under_sampling import ClusterCentroids, NearMiss
from scipy.optimize import differential_evolution
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, HDBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier, NearestNeighbors
from sklearn.inspection import permutation_importance
from scipy.spatial.distance import euclidean
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
#from ctgan import CTGAN

### Read datasets after preprocessed

In [29]:
#oversampling data
original_df = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\encoded_normalized\\original_data_normalized.csv")
mix_data_df = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\encoded_normalized\\mix_data_normalized.csv")
smote_df = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\encoded_normalized\\smote_data_normalized.csv") 
gan_df = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\encoded_normalized\\GAN_data_normalized.csv")
borderline_df = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\encoded_normalized\\borderline_data_normalized.csv")

X_test = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\test\\X_test_norm.csv")

In [30]:
#Before undersampling
print(original_df["target"].value_counts())
count1=original_df["target"].value_counts().sum()
count2=abs((original_df['target']==0).sum() - (original_df['target']==1).sum())
print(f"Before undersampling: {count2}")
print(f"After number of samples: {count1+count2}")

target
0    151
1     46
Name: count, dtype: int64
Before undersampling: 105
After number of samples: 302
